# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symmer.utils import unit_n_sphere_cartesian_coords
from symmer.symplectic import PauliwordOp, QuantumState, array_to_QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp.from_list(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.000+0.000j |0> ->  0.000+1.000j |1>
 1.000+0.000j |1> ->  0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp.from_list(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.000+0.000j |00>

After application of the Hadamard gate:

 0.500+0.000j |00> +
 0.500+0.000j |01> +
 0.500+0.000j |10> +
 0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

 0.500+0.000j ZZ +
 0.500+0.000j ZX +
 0.500+0.000j XZ +
 0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.083+0.368j |0000010101> +
 0.280+0.208j |0101011011> +
 0.255+0.342j |0101000100> +
 0.208+0.035j |1100000000> +
 0.116+0.283j |0001000001> +
 0.071+0.263j |0111010111> +
 0.122+0.001j |0010001101> +
 0.014+0.017j |0111101100> +
 0.152+0.247j |1111110101> +
 0.313+0.379j |0000010111>

 0.044+0.198j |1100101101> +
 0.072+0.020j |1010001011> +
 0.088+0.235j |1010100001> +
 0.314+0.220j |0010010001> +
 0.228+0.236j |0100111111> +
 0.278+0.217j |0110101001> +
 0.164+0.297j |1011000000> +
 0.144+0.201j |1111110000> +
 0.105+0.383j |1000001111> +
 0.186+0.378j |0111001011>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.058+0.260j |0000010101> +
 0.221+0.268j |0000010111> +
 0.082+0.200j |0001000001> +
 0.086+0.001j |0010001101> +
 0.222+0.156j |0010010001> +
 0.162+0.167j |0100111111> +
 0.181+0.242j |0101000100> +
 0.198+0.147j |0101011011> +
 0.197+0.154j |0110101001> +
 0.131+0.267j |0111001011> +
 0.050+0.186j |0111010111> +
 0.010+0.012j |0111101100> +
 0.074+0.270j |1000001111> +
 0.051+0.014j |1010001011> +
 0.063+0.166j |1010100001> +
 0.116+0.210j |1011000000> +
 0.147+0.025j |1100000000> +
 0.031+0.140j |1100101101> +
 0.102+0.142j |1111110000> +
 0.108+0.175j |1111110101>

Norm: (1+0j)


In [8]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

-0.696+0.000j XIXXYYYIZX +
 8.785+0.000j ZZZIIXZYIY +
-0.736+0.000j YYXIXZYZIY +
 1.922+0.000j XZXYZYIZZX +
 0.965+0.000j XYIXXXZYYZ +
 1.037+0.000j XZXZIZZZII +
 0.520+0.000j ZIIIIYYIIY +
 1.359+0.000j XIIZZZIYIY +
 0.114+0.000j XYYYYIXZXZ +
 1.437+0.000j ZZZIYZIIXX

Expectation value <psi|op|psi> = (0.006278384835572689+0j)


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [9]:
statevector = np.zeros(2**10)
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = array_to_QuantumState(statevector)
print(psi)
print(psi.conjugate * psi)

 0.408+0.000j |0000000000> +
 0.577+0.000j |0011111010> +
 0.707+0.000j |0111110100>
(1+0j)


In [10]:
PauliwordOp.from_matrix(np.array([[1,1],[1,-1]])) * (1/np.sqrt(2)) * QuantumState([[0]])

 0.707+0.000j |0> +
 0.707+0.000j |1>